In [1]:
# esta celda controla el estilo del cuaderno
from IPython.core.display import HTML
def css_styling():
    styles = open("custom.css", "r").read()
    return HTML(styles)
css_styling()

# Librerías de PLN en Python

## Introduccción

Como ocurre en otras áreas relacionadas con la Ciencia de Datos y el Aprendizaje Automático, Python es probablemente el lenguaje de programación que tiene disponibles más herramientas para realizar tareas de Procesamiento del Lenguaje Natural. En esta nota técnica vamos a hablar de tres de ellas: [spaCy](http://www.spacy.io/), [flair](https://github.com/flairNLP/flair) y [transformers](https://huggingface.co/transformers/). 


## spaCy

[spaCy](http://www.spacy.io/), de la empresa [Explosion](https://explosion.ai/), es una librería de procesamiento del lenguaje natural, de alto nivel, robusta, rápida, fácil de instalar y utilizar, e integrable con [otras librerías de *NLP* y de *deep learning*](https://spacy.io/universe). 

Tiene modelos entrenados en varios idiomas y permite realizar las [típicas tareas](https://spacy.io/usage/facts-figures) de segmentación por oraciones, tokenizanción, análisis morfológico, extracción de entidades y análisis de opinión. Vamos a aprender a utilizarla.

### Instalamos las dependencias

Lo primero que vamos a hacer es instalar la librería y [descargar un par de modelos](https://spacy.io/usage/models) que ya están entrenados y que nos van a permiter realizar tareas de PLN en inglés y español.

<div class="alert alert-warning">
  <p>Atención: Si estás ejecutando este cuaderno en local, asegúrate de que estás usando un entorno virtual.</p>
</div>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# instalamos spacy
!pip install -U spacy

# y descargamos un par de modelos pre-entrenados para inglés y español
!python -m spacy download es_core_news_md
!python -m spacy download en_core_web_md

### Segmentación y tokenización con spaCy

Para poder utilizar spaCy, necesitamos importar la librería y crear un objeto de tipo analizador con alguno de los modelos instalados. Comencemos cargando el modelo en español.

In [3]:
import spacy

# creamos un analizador con el modelo entrenado en español
nlp_es = spacy.load("es_core_news_md")

Para analizar cualquier texto en lenguaje natural, basta con crear un objeto de tipo documento pasándole una cadena de texto analizador.

In [4]:
texto = """Premio Nobel de Química 2020 para Charpentier y Doudna por CRISPR. La Real Academia Sueca de Ciencias otorga el Nobel de Química a Emmanuelle Charpentier y Jennifer A. Doudna por el desarrollo de esta herramienta de edición genética. El español Francis Mojica se queda sin reconocimiento a pesar de haber descubierto el sistema CRISPR en bacterias."""

# y procesamos el texto
doc = nlp_es(texto)

Verás que el paso anterior no produce ninguna salida aparente. Pero ese objeto `doc` que acabamos de crear en realidad contiene mucha información resultado de ejecutar varias tareas de comprensión del lenguaje natural sobre el contenido de `texto`. En una sola línea de código hemos segmentado el texto en oraciones, hemos tokenizado las oraciones, hemos realizado el análisis morfo-sintáctico de las oraciones y hemos hecho reconocimiento de entidades.

Veamos a continuación cómo podemos acceder a esa información de manera programática.

Podemos iterar sobre las oraciones del documento a través de la propiedad `doc.sents`.

In [5]:
for oracion in doc.sents:
    print(f"{oracion}\n")

Premio Nobel de Química 2020 para Charpentier y Doudna por CRISPR.

La Real Academia Sueca de Ciencias otorga el Nobel de Química a Emmanuelle Charpentier y Jennifer A. Doudna por el desarrollo de esta herramienta de edición genética.

El español Francis Mojica se queda sin reconocimiento a pesar de haber descubierto el sistema CRISPR en bacterias.



Cada oración está a su vez segmentada en unidades indivisibles llamados *tokens*. En este caso de spaCy, los tokens coinciden con la idea intuitiva de lo que es una palabra o un signo de puntuación.

In [6]:
for n, oracion in enumerate(doc.sents):
    print(f"La oración {n+1} tiene {len(oracion)} tokens.")

La oración 1 tiene 12 tokens.
La oración 2 tiene 28 tokens.
La oración 3 tiene 19 tokens.


Para verlo más claro y poder reutilizar el código, vamos a crear una sencilla función para tokenizar texto usando spaCy.

In [7]:
from typing import List

def tokenize(text: str, model: spacy.language.Language) -> List[str]:
    """Returns a tokenized version of a text using a spaCy model"""
    return [t.text for t in model(text)]

print(tokenize("¿Esto es solo una texto de ejemplo? Mmm... ¡Pues parece que sí 😃!", model=nlp_es))

['¿', 'Esto', 'es', 'solo', 'una', 'texto', 'de', 'ejemplo', '?', 'Mmm', '...', '¡', 'Pues', 'parece', 'que', 'sí', '😃', '!']


Y si probamos a tokenizar varias oraciones de ejemplo, verás que efectivamente los tokens coinciden con la idea intuitiva que tenemos en español de lo que es una palabra, un emoji o un signo de puntuación.

In [8]:
for oracion in doc.sents:
    print(tokenize(oracion.text, model=nlp_es), "\n")

['Premio', 'Nobel', 'de', 'Química', '2020', 'para', 'Charpentier', 'y', 'Doudna', 'por', 'CRISPR', '.'] 

['La', 'Real', 'Academia', 'Sueca', 'de', 'Ciencias', 'otorga', 'el', 'Nobel', 'de', 'Química', 'a', 'Emmanuelle', 'Charpentier', 'y', 'Jennifer', 'A.', 'Doudna', 'por', 'el', 'desarrollo', 'de', 'esta', 'herramienta', 'de', 'edición', 'genética', '.'] 

['El', 'español', 'Francis', 'Mojica', 'se', 'queda', 'sin', 'reconocimiento', 'a', 'pesar', 'de', 'haber', 'descubierto', 'el', 'sistema', 'CRISPR', 'en', 'bacterias', '.'] 



Más adelante veremos cómo es esta tokenización en inglés.

### Análisis morfo-sintáctico son spaCy

El documento que hemos analizado contiene completa información morfo-sintáctica de todos los tokens. ¿Qué tipo de información es esta? Veamos un ejemplo sobre una de las oraciones y [revisa la documentación del API de spaCy](https://spacy.io/api/token).

In [9]:
tercera_oracion = list(doc.sents)[2]

for token in tercera_oracion:
    print(f"""El token "{token}" ocupa la posición {token.i}, entre los índices {token.idx} y {token.idx+len(token.text)}.""")
    print(f"""Tiene como forma canónica la forma "{token.lemma_}".""")
    print(f"""Es una palabra de tipo {token.pos_} con los siguientes rasgos morfológicos: {token.tag_}.""")
    print(f"""Su función en la oración es {token.dep_}.\n""")

El token "El" ocupa la posición 40, entre los índices 234 y 236.
Tiene como forma canónica la forma "El".
Es una palabra de tipo DET con los siguientes rasgos morfológicos: DET__Definite=Def|Gender=Masc|Number=Sing|PronType=Art.
Su función en la oración es det.

El token "español" ocupa la posición 41, entre los índices 237 y 244.
Tiene como forma canónica la forma "español".
Es una palabra de tipo NOUN con los siguientes rasgos morfológicos: NOUN__Gender=Masc|Number=Sing.
Su función en la oración es nsubj.

El token "Francis" ocupa la posición 42, entre los índices 245 y 252.
Tiene como forma canónica la forma "Francis".
Es una palabra de tipo PROPN con los siguientes rasgos morfológicos: PROPN.
Su función en la oración es appos.

El token "Mojica" ocupa la posición 43, entre los índices 253 y 259.
Tiene como forma canónica la forma "Mojica".
Es una palabra de tipo PROPN con los siguientes rasgos morfológicos: PROPN.
Su función en la oración es flat.

El token "se" ocupa la posición 4

En la celda anterior hemos impreso por pantalla toda la información relevante para describir los tokens de una de las oraciones. Estamos imprimiendo la posición y los *offsets* del token, el tipo de palabra o categoría gramatical a la que pertenece, su lema (la forma canónica bajo la que aparece en el diccionario), los rasgos morfológicos asociados (dependiendo del tipo de palabra, imprimimos el género y el número, o la persona y el tiempo, etc.) y la función que desempeña en la oración. 

No pretendo que conozcas el significado de toda la información que aparece, aunque algunas cosas sí es posible que te suenen de las clases de lengua. Por ejemplo, etiquetas como `DET`, `NOUN`, `VERB`, y `PROPN` se utilizan para describir respectivamente determinantes, nombres comunes, verbos y nombres propios. Los rasgos `Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part` describen un token como forma verbal de tipo participio, género másculino, número singular, tiempo pasado.

Sí es importante que sepas que estas etiquetas están diseñadas para describir la morfología y la sintaxis de muchas lenguas del mundo y que siguen las convenciones habituales en el ámbito de la lingüística y el PLN que se recogen bajo el proyecto [Universal Dependencies](https://universaldependencies.org/guidelines.html).

### Reconocimiento de entidades con spaCy

El texto que hemos analizado contenía varias entidades nombradas, esto es menciones a personas, organizaciones y lugares. Te en cuenta que algunas menciones pueden tener más de un token. Veamos cómo podemos acceder a ellas a través de la propiedad `doc.ents`.

In [10]:
print(f"El documento analizado contiene {len(doc.ents)} entidades.\n")

for n, entidad in enumerate(doc.ents):
    print(f"""  {n+1}. La mención "{entidad}" es de tipo {entidad.label_}, y ocupa los tokens {entidad.start}-{entidad.end}.""")

El documento analizado contiene 10 entidades.

  1. La mención "Premio Nobel de Química 2020" es de tipo MISC, y ocupa los tokens 0-5.
  2. La mención "Charpentier" es de tipo PER, y ocupa los tokens 6-7.
  3. La mención "Doudna" es de tipo PER, y ocupa los tokens 8-9.
  4. La mención "CRISPR" es de tipo ORG, y ocupa los tokens 10-11.
  5. La mención "Real Academia Sueca de Ciencias" es de tipo ORG, y ocupa los tokens 13-18.
  6. La mención "Nobel de Química" es de tipo ORG, y ocupa los tokens 20-23.
  7. La mención "Emmanuelle Charpentier" es de tipo PER, y ocupa los tokens 24-26.
  8. La mención "Jennifer A. Doudna" es de tipo PER, y ocupa los tokens 27-30.
  9. La mención "Francis Mojica" es de tipo PER, y ocupa los tokens 42-44.
  10. La mención "CRISPR" es de tipo MISC, y ocupa los tokens 55-56.


El [modelo en español que estamos utilizando](https://spacy.io/models/es#es_core_news_md) es capaz de identificar solo cuatro tipos de entidades distintos: personas (`PER`), organizaciones (`ORG`), lugares (`LOC`) y entidades de otros tipos que se engloban dentro de la categoría `MISC`.

La propia librería contiene [varias maneras de visualizar el análisis del documento](https://spacy.io/usage/visualizers), como se muestra en el reconocimiento de entidades de la celda siguiente.

In [11]:
from spacy import displacy

displacy.serve(doc, style="ent")


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


### Usando spaCy con otras lenguas

Hasta ahora hemos utilizado un modelo que ha sido entrenado con colleciones de datos en español. No solo eso, si profundizas en la documentación de los modelos disponibles en spaCy, descubrirás que [este modelo se llama precisamente `es_core_news_md`](https://spacy.io/models/es#es_core_news_md) porque está entrenado con datos en español (`es`), sobre colecciones de noticias (`news`) y de los tres tamaños disponibles este es el mediano (`md`).

El tamaño del modelo determina el rendimiento final. A *grosso modo*, cuanto más grande sea, mejor calidad tendrá, aunque será más pesado y complejo de manejar. El origen de los datos también hay que tenerlo en cuenta. Los textos provenientes de medios de comunicación de masas y noticias contienen ejemplos de lengua en su variante culta. Esto significa que funcionarán muy bien para analizar texto de las mismas características, pero que probablemente funcionen mal si queremos procesar textos que usen la variante coloquial de la lengua o registros no estándar, que es lo que ocurre en redes sociales o con la mensajería instatánea. Mucho cuidado con usar modelos de PLN entrenados con noticias para tratar analizar mensajes de chat entre adolescentes. No van a a funcionar bien.

En cualquier caso, en spaCy tenemos disponibles otros modelos entreandos para otras lenguas. Por ejemplo, el modelo que cargamos a continuación es de tamaño mediano (`md`), está entrenado con datos en inglés (`en`), con documentos provenientes de la páginas web, blogs, y comentarios de usuarios (`web`). *A priori*, este modelo debería funcionar mejor para analizar lengua no culta.

In [12]:
nlp_en = spacy.load("en_core_web_md")

El mismo código que hemos usado en las celdas anteriores sirve para procesar un texto en inglés.

In [13]:
text = """A trade war between the world's two largest economies officially began on Friday morning. The Trump administration followed through with its threat to impose tariffs on $34 billion worth of Chinese products, a significant escalation of a fight that could hurt companies and consumers in both the United States and China."""

# y procesamos el texto
doc_en = nlp_en(text)

Podemos segmentar en oraciones y tokens.

In [14]:
for oracion in doc_en.sents:
    print(f"{oracion}\n")

A trade war between the world's two largest economies officially began on Friday morning.

The Trump administration followed through with its threat to impose tariffs on $34 billion worth of Chinese products, a significant escalation of a fight that could hurt companies and consumers in both the United States and China.



In [15]:
for n, oracion in enumerate(doc_en.sents):
    print(f"La oración {n+1} tiene {len(oracion)} tokens.")

La oración 1 tiene 16 tokens.
La oración 2 tiene 40 tokens.


Vamos a recuperar la función para tokenizar texto que hemos definido más arriba. ¿Cómo son los tokens que segmenta spaCy cuando la lengua es el inglés?

In [16]:
for oracion in doc_en.sents:
    print(tokenize(oracion.text, model=nlp_en), "\n")

['A', 'trade', 'war', 'between', 'the', 'world', "'s", 'two', 'largest', 'economies', 'officially', 'began', 'on', 'Friday', 'morning', '.'] 

['The', 'Trump', 'administration', 'followed', 'through', 'with', 'its', 'threat', 'to', 'impose', 'tariffs', 'on', '$', '34', 'billion', 'worth', 'of', 'Chinese', 'products', ',', 'a', 'significant', 'escalation', 'of', 'a', 'fight', 'that', 'could', 'hurt', 'companies', 'and', 'consumers', 'in', 'both', 'the', 'United', 'States', 'and', 'China', '.'] 



Quizá lo más llamativo es que en inglés se tokenizan algunas formas verbales que llevan contracciones, los genitivos sajón y algunos símbolos.

In [17]:
print(tokenize("You'd better go with those $1000! You aren't as smart as you think.", model=nlp_en))

['You', "'d", 'better', 'go', 'with', 'those', '$', '1000', '!', 'You', 'are', "n't", 'as', 'smart', 'as', 'you', 'think', '.']


Como hemos visto antes, el objeto `doc_en` también contiene el análisis morfo-sintáctico del texto. Imprimamos el de la primera.

In [18]:
primera_oracion = list(doc_en.sents)[0]

for token in primera_oracion:
    print(f"""El token "{token}" ocupa la posición {token.i}, entre los índices {token.idx} y {token.idx+len(token.text)}.""")
    print(f"""Tiene como forma canónica la forma "{token.lemma_}".""")
    print(f"""Es una palabra de tipo {token.pos_} con los siguientes rasgos morfológicos: {token.tag_}.""")
    print(f"""Su función en la oración es {token.dep_}.\n""")

El token "A" ocupa la posición 0, entre los índices 0 y 1.
Tiene como forma canónica la forma "a".
Es una palabra de tipo DET con los siguientes rasgos morfológicos: DT.
Su función en la oración es det.

El token "trade" ocupa la posición 1, entre los índices 2 y 7.
Tiene como forma canónica la forma "trade".
Es una palabra de tipo NOUN con los siguientes rasgos morfológicos: NN.
Su función en la oración es compound.

El token "war" ocupa la posición 2, entre los índices 8 y 11.
Tiene como forma canónica la forma "war".
Es una palabra de tipo NOUN con los siguientes rasgos morfológicos: NN.
Su función en la oración es nsubj.

El token "between" ocupa la posición 3, entre los índices 12 y 19.
Tiene como forma canónica la forma "between".
Es una palabra de tipo ADP con los siguientes rasgos morfológicos: IN.
Su función en la oración es prep.

El token "the" ocupa la posición 4, entre los índices 20 y 23.
Tiene como forma canónica la forma "the".
Es una palabra de tipo DET con los siguien

Si te fijas en la salida de este análisis y lo comparas con el del español, es probable que te llamen la atención varias cosas.

- El análisis no contiene rasgos morfológicos. Esto ocurre porque el inglés tiene una variación morfológica muy pobre (comparada con el español y otras lenguas). Si lo piensas un poco, los nombres en inglés no tienen marca de género, los adjetivos no distinguen entre singular y plural, y la conjugación de los verbos es muy sencilla.

- El análisis morfo-sintáctico en inglés muestra otras etiquetas. Ahora los determinantes, los nombres, los nombres propios y los vebos se anotan como `DT`, `NN` o `NNS`, `NNP` y `VB*`. ¿Por qué pasa esto? Precisamente, por las características de la lengua inglesa, tiene sentido utilizar un esquema específico adaptado a su vocabulario. El modelo en inglés que estamos usando ha sido entrenado de manera supervisada con datos que contienen otro tipo de anotaciones. Si tienes curiosidad en profundizar en este otro *tagset*, el conjunto de etiquetas que aquí aparece es muy conocido en el mundo del PLN porque fue el primero, y se denomina [etiquetas del corpus de Brown](https://en.wikipedia.org/wiki/Brown_Corpus#Part-of-speech_tags_used).

Por último, también podemos extraer y visualizar las entidades nombradas.

In [19]:
print(f"El documento analizado contiene {len(doc_en.ents)} entidades.\n")

for n, entidad in enumerate(doc_en.ents):
    print(f"""  {n+1}. La mención "{entidad}" es de tipo {entidad.label_}, y ocupa los tokens {entidad.start}-{entidad.end}.""")

El documento analizado contiene 7 entidades.

  1. La mención "two" es de tipo CARDINAL, y ocupa los tokens 7-8.
  2. La mención "Friday morning" es de tipo TIME, y ocupa los tokens 13-15.
  3. La mención "Trump" es de tipo ORG, y ocupa los tokens 17-18.
  4. La mención "$34 billion" es de tipo MONEY, y ocupa los tokens 28-31.
  5. La mención "Chinese" es de tipo NORP, y ocupa los tokens 33-34.
  6. La mención "the United States" es de tipo GPE, y ocupa los tokens 50-53.
  7. La mención "China" es de tipo GPE, y ocupa los tokens 54-55.


In [20]:
from spacy import displacy

displacy.serve(doc_en, style="ent")


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


Aquí también hay algunas diferencias que habrás notado con respecto al modelo en español. En este caso, el modelo utiliza otros identificadores diferentes para las entidades y es capaz de reconocer más tipos de entidades diferentes. Y en general, funciona mejor que el del español. Esto ocurre básicamente porque [este modelo en inglés](https://spacy.io/models/en#en_core_web_md), tal y como se describe en la documentación de spaCy, ha sido entrenado con un dataset más grande llamado [OntoNotes](https://catalog.ldc.upenn.edu/LDC2013T19).

<div class="alert alert-warning">
  <p>Como en otros ámbitos que utilizan aproximaciones basadas en aprendizaje automático y <i>deep learning</i>, el rendikiento de un sistema depende más de los datos que del algoritmo que hay detrás. En este caso, los datasets disponibles para el inglés son notablemente mayores y de mayor calidad que los disponibles en otras lenguas.</p>
</div>

## flair

[flair](https://github.com/flairNLP/flair) es una librería de PLN de software libre desarrollada inicialmente por la división de ingeniería de Zalando y basada en PyTorch. Permite acceder a funcionalidades muy interesantes para procesar lenguaje natural, algunas de ellas con aproximaciones muy modernas:

- segmentar el texto y [realizar operaciones básicas con lenguaje natural](https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_1_BASICS.md)
- etiquetar texto en lenguaje natural con modelos preentrenados con [información morfo-sintáctica, de entidades nombradas](https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_2_TAGGING.md)
- clasificar automáticamente texto
- entrenar tus propios modelos para [construir otros clasificadores](https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md)
- [representar las palabras como *embeddings*](https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_3_WORD_EMBEDDING.md) de distintos tipos

Veamos cómo podemos acceder a algunas de sus funcionalidades, pero lo primero es asegurarnos de que tenemos la librería instalada.

In [ ]:
# instalamos flair
!pip install -U flair

### Análisis morfológico con flair

El API de flair es diferente al que hemos visto en spaCy. En lugar de importar un único modelo pre-entrenado capaz de maneajar todos los aspectos de una lengua, con flair tendremos que instanciar determinadas clases para representar las oraciones y el modelo que queremos utilizar, en función del tipo de tarea de PLN que queramos resolver. 

Por ejemplo, para analizar sintácticamente un texto con flair, necesitamos es importar dos cosas: 

1. una  la clase `Sentence`, que nos permite representar una secuencia de texto en lenguaje natural con sentido completo, 
2. como la tarea que queremos resolver implica procesar y etiquetar secuencias ordenadas de tokens, necesitamos importar un `SequenceTagger` y construirlo con un modelo específico capaz de manejar información morfológica. 

[Hay varios modelos disponibles](https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_2_TAGGING.md#list-of-pre-trained-sequence-tagger-models), pero en este ejemplo vamos a utilizar uno pre-entrenado para reconocer las categorías gramaticales en varios idiomas.

In [22]:
from flair.data import Sentence
from flair.models import SequenceTagger

# cargamos el analizador con un modelo multi-idioma en su versión fast, que funciona en CPU
# la primera vez que intentas cargar un modelo tendrás que descargalo 
tagger = SequenceTagger.load("pos-multi-fast")

2020-11-01 18:53:21,342 loading file /home/victor/.flair/models/pos-multi-fast.pt


<div class="alert alert-warning">
  <p>Si tienes acceso a GPU, a través de <a href="https://colab.research.google.com/" target="_blank">Google Colab</a>, por ejemplo, no dudes en utilizar la versión completa y cargar el modelo llamado<code>pos-multi</code>.</p>
</div>

In [23]:
oracion_es = Sentence("Facebook nació hace década y media tras una noche de copas de Mark Zuckerberg.")
tagger.predict(oracion_es)
# imprimimos el análisis
print(oracion_es.to_tagged_string(), "\n")

oracion_fr = Sentence("Grand débat national: suivez Emmanuel Macron en direct de Bordeaux.")
tagger.predict(oracion_fr)
# imprimimos el análisis
print(oracion_fr.to_tagged_string(), "\n")

oracion_de = Sentence("Hier an der Zufahrt zur Startrampe 39A, wo vor 50 Jahren die gigantischen Saturn-Raketen der Apollo-Mondmissionen im Schneckentempo vorbeigefahren sind, prangen nun die blauen Lettern des Raumfahrtunternehmens von Elon Musk an einem Hangar.")
tagger.predict(oracion_de)
# imprimimos el análisis
print(oracion_de.to_tagged_string(), "\n")

Facebook <PROPN> nació <VERB> hace <VERB> década <NOUN> y <CCONJ> media <NOUN> tras <ADP> una <DET> noche <NOUN> de <ADP> copas <NOUN> de <ADP> Mark <PROPN> Zuckerberg <PROPN> . <PUNCT> 

Grand <ADJ> débat <NOUN> national <ADJ> : <PUNCT> suivez <VERB> Emmanuel <PROPN> Macron <PROPN> en <ADP> direct <NOUN> de <ADP> Bordeaux <PROPN> . <PUNCT> 

Hier <ADV> an <ADP> der <DET> Zufahrt <NOUN> zur <ADP> Startrampe <NOUN> 39A <PROPN> , <PUNCT> wo <ADV> vor <ADP> 50 <NUM> Jahren <NOUN> die <DET> gigantischen <ADJ> Saturn-Raketen <NOUN> der <DET> Apollo-Mondmissionen <NOUN> im <ADJ> Schneckentempo <NOUN> vorbeigefahren <VERB> sind <AUX> , <PUNCT> prangen <VERB> nun <ADV> die <DET> blauen <ADJ> Lettern <NOUN> des <DET> Raumfahrtunternehmens <NOUN> von <ADP> Elon <PROPN> Musk <PROPN> an <ADP> einem <DET> Hangar <NOUN> . <PUNCT> 



La salida de este etiquetador morfológico es bastante sencilla, solo imprime el nombre de la categoría gramatical, pero este modelo es capaz de reconocer clases de palabras en varias lenguas. Si te fijas, las etiquetas son las mismas que las utilizadas en [Universal Dependencies](https://universaldependencies.org/).

## Reconocimiento de entidades con flair

El reconocimiento de entidades también es una tarea que implica procesar un texto como una secuencia ordenada de tokens. En este caso, necesitamos instanciar otro `SequenceTagger` con el modelo específico para reconocer entidades en inglés.

In [24]:
# cargamos el modelo entrenado para reconocer de entidades
# recuerda, NER -> named entities recognition
tagger = SequenceTagger.load("ner-fast")

2020-11-01 18:53:23,302 loading file /home/victor/.flair/models/en-ner-fast-conll03-v0.4.pt


In [25]:
# analizamos una oración
oracion = Sentence("""Behind closed doors, freshman Rep. Alexandria Ocasio Cortez threatened the Senate of the United States to put those voting with Republicans "on a list" for a primary challenge in the 2020 election.""")
tagger.predict(oracion)
# imprimimos el análisis
print(oracion.to_tagged_string())

Behind closed doors , freshman Rep <S-MISC> . Alexandria <B-PER> Ocasio <I-PER> Cortez <E-PER> threatened the Senate <S-ORG> of the United <B-LOC> States <E-LOC> to put those voting with Republicans <S-MISC> " on a list " for a primary challenge in the 2020 election .


La prestas atención a la salida impresa de este análisis, verás que contiene los tokens de la oración y solo aquellas menciones identificadas como entidades nombradas imprimirán información adicional. Pero, ¿cómo se codifica e interpreta esta salida?

El esquema de anotación que utiliza flair en este ejemplo es bastante habitual en tareas de reconocimiento de entidades y se denomina [esquema IOB (Inside-Outside-Beginning)](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)). 

Como ya sabes, un reconocedor de entidades generalista normalmente identificará varios tipos de entidades diferentes: personas (`PER`), organizaciones (`ORG`), lugares (`LOC`) y otras entidades (`MISC`). Y como hemos visto antes, las menciones de las entidades nombradas pueden expandirse a lo largo de más de un token. Pues bien, este esquema de anotación especifica con prefijos si el token en cuestión es el primer elemento de la mención (*beginning*: `B-`), si es un token dentro de la mención (*inside*: `I-`), si es el último elemento de la mención (*end*: `E-`), o si la entidad tiene un solo token (*single*: `S-`). De este modo, `Alexandria <B-PER> Ocasio <I-PER> Cortez <E-PER>` permite describir una entidad de tipo persona con tres tokens, `United <B-LOC> States <E-LOC>` es una entidad de tipo lugar con dos tokens, y `Republicans <S-MISC>` es una entidad de tipo indeterminado formada por un solo token.

De manera programática, puedes acceder a la información completa de las entidades como se muestra en el siguiente ejemplo.

In [26]:
# iteramos por la entidades
for entidad in oracion.get_spans("ner"):
    print(f"""La entidad "{entidad.text}" ocupa los tokens {[t.idx for t in entidad.tokens]} y los offsets {entidad.start_pos} y {entidad.end_pos}.""")
    print(f"""Pertenece a la categoria {entidad.tag} con una probabilidad de {entidad.score}.\n""")

La entidad "Rep" ocupa los tokens [6] y los offsets 30 y 33.
Pertenece a la categoria MISC con una probabilidad de 0.9307801723480225.

La entidad "Alexandria Ocasio Cortez" ocupa los tokens [8, 9, 10] y los offsets 35 y 59.
Pertenece a la categoria PER con una probabilidad de 0.9782136678695679.

La entidad "Senate" ocupa los tokens [13] y los offsets 75 y 81.
Pertenece a la categoria ORG con una probabilidad de 0.9942795038223267.

La entidad "United States" ocupa los tokens [16, 17] y los offsets 89 y 102.
Pertenece a la categoria LOC con una probabilidad de 0.9732602536678314.

La entidad "Republicans" ocupa los tokens [23] y los offsets 128 y 139.
Pertenece a la categoria MISC con una probabilidad de 0.9998949766159058.



In [27]:
# o imprimimos la estructura de datos con el análisis completo
print(oracion.to_dict(tag_type="ner"))

{'text': 'Behind closed doors, freshman Rep. Alexandria Ocasio Cortez threatened the Senate of the United States to put those voting with Republicans "on a list" for a primary challenge in the 2020 election.', 'labels': [], 'entities': [{'text': 'Rep', 'start_pos': 30, 'end_pos': 33, 'labels': [MISC (0.9308)]}, {'text': 'Alexandria Ocasio Cortez', 'start_pos': 35, 'end_pos': 59, 'labels': [PER (0.9782)]}, {'text': 'Senate', 'start_pos': 75, 'end_pos': 81, 'labels': [ORG (0.9943)]}, {'text': 'United States', 'start_pos': 89, 'end_pos': 102, 'labels': [LOC (0.9733)]}, {'text': 'Republicans', 'start_pos': 128, 'end_pos': 139, 'labels': [MISC (0.9999)]}]}


## Análisis de opinión con flair

Otra interesante característica de flair es que nos da acceso a modelos entrenados con información sobre análisis de opinión, que permiten detectar opiniones positivas y negativas.

En este caso, la tarea de análisis de opinión es un problema de claficación de texto, no de etiquetado por secuencia. Necesitaremos instanciar un objeto diferente: un `TextClassifier`, como se muestra en el siguiente ejemplo.

In [28]:
from flair.models import TextClassifier

classifier = TextClassifier.load("en-sentiment")

2020-11-01 18:53:26,103 loading file /home/victor/.flair/models/sentiment-en-mix-distillbert_3.1.pt


In [29]:
# definimos unas cuantas oraciones en inglés
oraciones = ["I love ice-cream!", "Don't ever go to this restaurant. The food was horrible :-(", "Evil Corp to announce job cuts.", "What a wonderful world!"]

for oracion in oraciones:
    s = Sentence(oracion)
    classifier.predict(s)
    polaridad, prob = s.labels[0].value, s.labels[0].score

    print(f"""La oración "{s.to_plain_string()}" """)
    print(f"contiene una opinión de tipo {polaridad} con una probabilidad de {prob:.2f}\n")

La oración "I love ice-cream!" 
contiene una opinión de tipo POSITIVE con una probabilidad de 1.00

La oración "Don't ever go to this restaurant. The food was horrible :-(" 
contiene una opinión de tipo NEGATIVE con una probabilidad de 1.00

La oración "Evil Corp to announce job cuts." 
contiene una opinión de tipo NEGATIVE con una probabilidad de 1.00

La oración "What a wonderful world!" 
contiene una opinión de tipo POSITIVE con una probabilidad de 1.00



## transformers

[transformers](https://github.com/huggingface/transformers), liderada por la empresa [HuggingFace](https://huggingface.co/), se ha convertido en los últimos años en una las librerías para tareas de NLU y NLG más importantes. En un primer momento surgió como un punto intermedio que permitía integrar en código de PyTorch modelos entrenados originalmente en TensorFlow. Actualmente, da acceso a [multitud de modelos pre-entrenados con un mismo API](https://huggingface.co/models) y está en continua evolución, dado que cuenta con una comunidad de usuarios muy activa. 

<div class="alert alert-info">
  <p>Si tienes curiosidad en profundizar, el nombre de esta librería hace referencia a <a href="https://jalammar.github.io/illustrated-transformer" target="_blank">Transformer</a>, una popular arquitectura de <em>deep learning</em> que utiliza <a href="https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/" target="_blank">un mecanismo denominado <strong>atención</strong></a>, y que ha revolucionado el mundo del procesamiento del lenguaje natural.</p>
</div>

La [documentación de transformers es muy completa](https://huggingface.co/transformers/) y el alcance de la herramienta es muy amplio. Para el objetivo de nuestra asignatura, nos contentaremos con entender cómo podemos utilizar los modelos con uno de los interfaces de más alto nivel: [los *pipelines*](https://huggingface.co/transformers/main_classes/pipelines.html). ¡Vamos allá!

Lo primero es instalar la librería.

In [ ]:
# instalamos flair
!pip install -U transformers


El objeto `pipeline` es una abstracción que nos permite especifcar la tarea de PLN y cargar cualquier modelo pre-entrenado que esté disponible en nuestra máquina local o en [el registro de modelos públicos de huggingFace](https://huggingface.co/models).

Podemos crear un *pipeline* de varias maneras pero la forma más sencilla es especificando el tipo de tarea y el modelo que queremos utilizar.

### Reconocimiento de entidades con transformers

Para realizar reconocimiento de entidades en español, necesitamos encontrar un modelo adecuado para la tarea y el idioma. Si buscamos [modelos pre-entrenados para NER en español](https://huggingface.co/models?filter=es&search=ner) encontraremos varios de ellos. Muy probablemente estos nombres no te dirán nada, pero no abrumes, te aseguro que siguen una nomenclatura bastante habitual en el mundillo. 

Por ejemplo, el modelo llamado [`mrm8488/bert-spanish-cased-finetuned-ner`](https://huggingface.co/mrm8488/bert-spanish-cased-finetuned-ner) lo ha entrenado un famoso miembro de la comunidad de HuggingFace llamado [`@mrm8488`](https://twitter.com/mrm8488), ajustando para reconocimiento de entidades (haciendo *fine tune* para NER en español) un modelo BERT que distingue mayúsculas (*cased*). En cualquier caso, siempre puedes [acceder a una pequeña ficha con información extra, métricas de evaluación y ejemplos de uso](https://huggingface.co/mrm8488/bert-spanish-cased-finetuned-ner).

Vamos a ver qué tal funciona.

In [31]:
from transformers import pipeline

ner_es_classifier = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner")

In [32]:
ner_es_classifier(texto)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'word': 'Premio',
  'score': 0.9974514245986938,
  'entity': 'B-MISC',
  'index': 1},
 {'word': 'Nobel',
  'score': 0.9970600008964539,
  'entity': 'I-MISC',
  'index': 2},
 {'word': 'de', 'score': 0.9981623888015747, 'entity': 'I-MISC', 'index': 3},
 {'word': 'Química',
  'score': 0.9982280731201172,
  'entity': 'I-MISC',
  'index': 4},
 {'word': '2020', 'score': 0.922611653804779, 'entity': 'I-MISC', 'index': 5},
 {'word': 'Char', 'score': 0.9955931305885315, 'entity': 'B-PER', 'index': 7},
 {'word': '##pent',
  'score': 0.6227113604545593,
  'entity': 'B-PER',
  'index': 8},
 {'word': '##ier', 'score': 0.5794926881790161, 'entity': 'B-PER', 'index': 9},
 {'word': 'Do', 'score': 0.9936509132385254, 'entity': 'B-PER', 'index': 11},
 {'word': '##ud', 'score': 0.5442298054695129, 'entity': 'I-PER', 'index': 12},
 {'word': '##na', 'score': 0.938393771648407, 'entity': 'I-PER', 'index': 13},
 {'word': 'CR', 'score': 0.9774543046951294, 'entity': 'B-MISC', 'index': 15},
 {'word': '##IS',

La salida del reconocedor de entidades es una lista de diccionarios normal de Python. Si te fijas con atención, cada elemento de la lista es un token, y estos tokens son radicalmente diferentes a los que hemos visto hasta ahora. Hay tokens que coinciden con unidades que llamaríamos palabras (*premio*, *Nobel* o *química*), pero otros tokens son secuencias menores, algunos de ellos sin sentido gramatical. Esto suele ocurrir con nombres propios que no son habituales en español y con palabras de reciente creación (*Charpentier* se tokeniza `Char` `##pent`, `##ier` ; *CRISPR* se tokeniza como `CR`, `##IS`, `##PR`), pero también podemos encontrar esta tokenización en palabras perfectamente españolas como *sueca* (`sue`,`##ca`). 

[Este tipo de tokenización tiene un porqué](https://huggingface.co/transformers/tokenizer_summary.html), pero por el momento, es suficiente con que entiendas que cuando un token es la continuación de una unidad subpalabra, se especifica con dos almohadillas (`##`).

In [33]:
for token in ner_es_classifier(texto):
    if token["entity"].endswith("PER"):
        print(f"""{token["word"]} es una mención a una persona con una probabilidad de {token["score"]:.2f}""")

Char es una mención a una persona con una probabilidad de 1.00
##pent es una mención a una persona con una probabilidad de 0.62
##ier es una mención a una persona con una probabilidad de 0.58
Do es una mención a una persona con una probabilidad de 0.99
##ud es una mención a una persona con una probabilidad de 0.54
##na es una mención a una persona con una probabilidad de 0.94
Emma es una mención a una persona con una probabilidad de 1.00
##nu es una mención a una persona con una probabilidad de 0.99
##elle es una mención a una persona con una probabilidad de 1.00
Char es una mención a una persona con una probabilidad de 1.00
##pent es una mención a una persona con una probabilidad de 1.00
##ier es una mención a una persona con una probabilidad de 1.00
[UNK] es una mención a una persona con una probabilidad de 1.00
A es una mención a una persona con una probabilidad de 1.00
. es una mención a una persona con una probabilidad de 1.00
Do es una mención a una persona con una probabilidad d

Para probar el reconocedor de entidades en inglés, es suficiente con que busquemos un modelo adecuado e instanciemos un *pipeline* con él.

In [34]:
ner_en_classifier = pipeline("ner", model="dslim/bert-base-NER")

In [35]:
ner_en_classifier(text)

[{'word': 'Trump',
  'score': 0.9974536895751953,
  'entity': 'B-PER',
  'index': 19},
 {'word': 'Chinese',
  'score': 0.9997501969337463,
  'entity': 'B-MISC',
  'index': 37},
 {'word': 'United',
  'score': 0.9997009634971619,
  'entity': 'B-LOC',
  'index': 57},
 {'word': 'States',
  'score': 0.9994767308235168,
  'entity': 'I-LOC',
  'index': 58},
 {'word': 'China',
  'score': 0.9998029470443726,
  'entity': 'B-LOC',
  'index': 60}]

### Análisis de opinión con transformers

Tenemos a nuestra disposición modelos de clasificación de texto entrenados para reconocer la polaridad de textos en inglés. Podemos crear un clasificador de sentimiento con el modelo por defecto y probar qué tal funciona con algunas frases.

In [36]:
# si no especificamos el modelo, utilizamos el definido por defecto para la tarea de sentiment analysis
sentiment_classifier = pipeline("sentiment-analysis")

In [37]:
for ejemplo in oraciones:
    print(f"""La frase "{ejemplo}" es """)
    print(sentiment_classifier(ejemplo), "\n")

La frase "I love ice-cream!" es 
[{'label': 'POSITIVE', 'score': 0.9998226165771484}] 

La frase "Don't ever go to this restaurant. The food was horrible :-(" es 
[{'label': 'NEGATIVE', 'score': 0.9981479644775391}] 

La frase "Evil Corp to announce job cuts." es 
[{'label': 'NEGATIVE', 'score': 0.9994521737098694}] 

La frase "What a wonderful world!" es 
[{'label': 'POSITIVE', 'score': 0.9998851418495178}] 



### Búsqueda de respuestas con transformers

Hasta ahora no habíamos podido probar tareas de PLN más complejas, pero transformers nos da acceso a modelos entrenados para tareas como la búsqueda de respuestas que tienen un rendimiento sorprendentemente bueno, al menos en inglés. Vamos a comprobar qué tal funcionan.

Lo primero es cargar el extractor de respuestas con el modelo por defecto en inglés.

In [38]:
qa = pipeline("question-answering")

A continuación definimos una serie de pares pregunta-contexto, con textos en lenguaje natural pegados directamente de artículos de Wikipedia.

El modelo de búsqueda de respuestas es de tipo extractivo, no generativo. Esto implica que el modelo espera procesar e interpretar una pregunta y un contexto, susceptible de contener la respuesta. Intentará interpretar el tipo de pregunta y devolverá el segmento del contexto con mayor probabilidad de ser la respuesta. Ten en cuenta que el modelo es extractivo, por lo tanto es incapaz de reescribir la respuesta.

Con estos ejemplos funciona perfectamente. Parece que hemos alcanzado la singularidad y que la humanidad está a merced de las máquinas.

In [39]:
preguntas_contextos = [
    {
        "question": "What city are the Lakers from?",
        "context": "The Los Angeles Lakers are an American professional basketball team based in Los Angeles."
    },
    {
        "question": "LeBron James date of birth",
        "context": "LeBron Raymone James Sr. (/ləˈbrɒn/ lə-BRON; born December 30, 1984) is an American professional basketball player for the Los Angeles Lakers of the National Basketball Association (NBA)."
    },
    {
        "question": "Who won the NBA finals?",
        "context": "The 2020 NBA Finals was the championship series of the National Basketball Association (NBA)'s 2019–20 season and conclusion of the season's playoffs. In this best-of-seven playoff series, the Western Conference champion Los Angeles Lakers defeated the Eastern Conference champion Miami Heat, 4–2, winning their first NBA championship in ten years."
    },
    {
        "question": "What was the score?",
        "context": "The 2020 NBA Finals was the championship series of the National Basketball Association (NBA)'s 2019–20 season and conclusion of the season's playoffs. In this best-of-seven playoff series, the Western Conference champion Los Angeles Lakers defeated the Eastern Conference champion Miami Heat, 4–2, winning their first NBA championship in ten years."
    }
]

for ejemplo in preguntas_contextos:
    print(ejemplo["question"])
    print(qa(ejemplo), "\n")

What city are the Lakers from?
{'score': 0.2602875530719757, 'start': 4, 'end': 15, 'answer': 'Los Angeles'} 

LeBron James date of birth
{'score': 0.9441075921058655, 'start': 50, 'end': 68, 'answer': 'December 30, 1984)'} 

Who won the NBA finals?
{'score': 0.2531222105026245, 'start': 221, 'end': 239, 'answer': 'Los Angeles Lakers'} 

What was the score?
{'score': 0.955236554145813, 'start': 293, 'end': 297, 'answer': '4–2,'} 



Sin embargo, cuando el modelo recibe un contexto no directamente relacionado con la pregunta, empieza a dudar y le vemos las vergüenzas. Claramente no entiende lo que está haciendo.

In [40]:
qa(
    {
        "question": "What are you from?",
        "context": "The New York Times (NYT), sometimes also known as The Times, is an American newspaper based in New York City with worldwide influence and readership"
    }
)

{'score': 0.3653542399406433, 'start': 67, 'end': 75, 'answer': 'American'}

In [41]:
qa(
    {
        "question": "What's the capital city of Spain?",
        "context": "Love is in the air, in the whisper of the trees. Love is in the air, in the thunder of the sea."
    }
)

{'score': 0.12368953973054886, 'start': 0, 'end': 4, 'answer': 'Love'}

### Resumen automático con transformers

Otra de las tareas que combinan NLU y NLP que tenemos disponible en transformers con muy pocas líneas de código es el resumen automático. Si creamos un *pipeline* para resumir, podremos procesar documentos largos de entrada y seleccionar aquellas frases que mejor resuman el contenido semántico del documento original.

Ten en cuenta que, en muchas ocasiones, a estos resúmenes les faltará la naturalidad, la coherencia y la cohesión típica de un resumen hecho a mano por seres humanos, pero el resultado sí que sintetizará el contenido de manera satisfactoria, lo que puede ser suficiente en muchos contextos.

In [42]:
summarizer = pipeline("summarization")

Una vez cargado el modelo, veamos qué tal funciona con textos extraídos de Wikipedia y de la prensa.

In [43]:
documento_largo = """The New York Times (NYT), sometimes also known as The Times,[6] is an American newspaper based in New York City with worldwide influence and readership.[7][8] Nicknamed "the Gray Lady",[9] the Times has long been regarded within the industry as a national "newspaper of record".[10] The paper's motto, "All the News That's Fit to Print", appears in the upper left-hand corner of the front page. Founded in 1851, the paper has won 130 Pulitzer Prizes, more than any other newspaper.[11][12] It is ranked 18th in the world by circulation and 3rd in the U.S.[13]

The paper is owned by The New York Times Company, which is publicly traded. It has been owned by the Sulzberger family since 1896 through a dual-class share structure.[14] A. G. Sulzberger and his father, Arthur Ochs Sulzberger Jr.—the paper's publisher and the company's chairman, respectively—are the fourth and fifth generation of the family to head the paper.[15]

Since the mid-1970s, The New York Times has greatly expanded its layout and organization, adding special weekly sections on various topics supplementing the regular news, editorials, sports, and features. Since 2008,[16] the Times has been organized into the following sections: News, Editorials/Opinions-Columns/Op-Ed, New York (metropolitan), Business, Sports, Arts, Science, Styles, Home, Travel, and other features.[17] On Sundays, the Times is supplemented by the Sunday Review (formerly the Week in Review),[18] The New York Times Book Review,[19] The New York Times Magazine,[20] and T: The New York Times Style Magazine.[21]

The Times stayed with the broadsheet full-page set-up and an eight-column format for several years after most papers switched to six,[22] and was one of the last newspapers to adopt color photography, especially on the front page.[23]
"""

summarizer(documento_largo)

[{'summary_text': ' The New York Times (NYT) is an American newspaper based in New York City . Founded in 1851, the paper has won 130 Pulitzer Prizes, more than any other newspaper . It is ranked 18th in the world by circulation and 3rd in the U.S.'}]

In [44]:
documento_largo_2 = """Roger Federer hailed his "greatest rival" after Rafael Nadal equalled his record of 20 Grand Slam men's singles titles in devastating fashion.

Nadal, 34, comprehensively outplayed the other member of the sport's 'Big Three', Novak Djokovic 6-0 6-2 7-5 to claim his 13th French Open title.

"I have always had the utmost respect for my friend Rafa as a person and as a champion," the Swiss great tweeted.

In response, Nadal thanked Federer and said their relationship "means a lot".

The Spaniard continued: "As everybody know we have a very, very good relationship and we respect each other a lot and at the same time I think he is happy when I am winning and I'm happy when he is doing things well.

"In some ways it means a lot, the positive relationship we have together because we have been going through a great rivalry for a very, very long time."

Federer, who missed Roland Garros this year after knee surgery, added: "I hope 20 is just another step on the continuing journey for both of us.

"As my greatest rival over many years, I believe we have pushed each other to become better players.

"Therefore, it is a true honour for me to congratulate him on his 20th Grand Slam victory. It is especially amazing that he has now won Roland Garros an incredible 13 times, which is one of the greatest achievements in sport.
"""

print(summarizer(documento_largo_2)[0]["summary_text"])

 Rafael Nadal equalled Roger Federer's record of 20 Grand Slam men's singles titles . Nadal defeated Novak Djokovic 6-0 6-2 7-5 to claim his 13th French Open title . Federer thanked Nadal and said their relationship "means a lot" Nadal: "I have always had the utmost respect for my friend Rafa"


## Conclusiones

En esta segunda nota técnica, hemos presentado tres de las librerías de Python más populares para realizar tareas de procesamiento del lenguaje natural: [spaCy](http://www.spacy.io/), [flair](https://github.com/flairNLP/flair) y [transformers](https://huggingface.co/transformers/).

Hemos aprendido cómo cargar modelos pre-entrenados que están disponibles en todas ellas para realizar tareas básicas de NLU como son la tokenización, el análisis morfo-sintáctico y el reconocimiento de entidades y tareas no tan básicas como la búsqueda de respuestas y el resumen automático.